In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from astropy.table import Table, vstack
import glob

## Format definition

In [ ]:
import matplotlib.style as style
style.use('seaborn-colorblind')

plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 20
plt.rcParams['figure.figsize']

def format_axes(ax,option):
    
    if(option=='width'):
        ax.set_xlabel('Muon radius [deg]')
        ax.set_ylabel('Muon ring width [deg]')
        ax.grid(linestyle="dashed")
        #ax.set_ylim(0.042,0.07)
        ax.legend()
    if(option=='size'):
        ax2.grid(linestyle="dashed")
        ax2.legend(ncol=1)
        ax2.set_xlabel('Muon ring radius [deg]')
        ax2.set_ylabel('Muon ring intensity [phe]')
        ax2.set_ylim(1800,3200)
        ax2.set_xlim(0.95,1.23)

## Definition of different MC efficiencies

In [ ]:
OPT_EFF=[100, 90, 110]
colors = ['C0', 'C1', 'C2']
labels = ['MC', 'MC (-10% optical efficiency)', 'MC (+10% optical efficiency)']
linestyles = ['solid', 'dashed', 'dashed']
facecolors = ['C0', 'none', 'none']

In [ ]:
# size (intensity) outside the ring, to get rid of hadronic showers
size_outside = 1 # phe

# intensity limit for muon rings
intensity_limit = 3000

# Define fitting function with origin at 0
from scipy.optimize import curve_fit
def func(x,a):
    return a*x

### Data

In [ ]:
listdir = glob.glob('data/Run2973/muons_LST-1.Run02973*')

dat = Table.read('{}'.format(listdir[0]), format='fits')
for i in range(1,len(listdir)):
    dat2 = Table.read('{}'.format(listdir[i]), format='fits')
    if(len(dat2) == 0):
        continue
    dat = vstack([dat, dat2])

df = dat.to_pandas()
df_good_data = df[df['good_ring']]

In [ ]:
radius_data = df_good_data['ring_radius'][(df_good_data['ring_completeness'] > 0.9) & (
    df_good_data['size_outside'] < size_outside) & (df_good_data['ring_width'] < 0.4) & (
    df_good_data['ring_size'] < intensity_limit)]
intensity_data = df_good_data['ring_size'][(df_good_data['ring_completeness'] > 0.9) & (
    df_good_data['size_outside'] < size_outside) & (df_good_data['ring_width'] < 0.4) & (
    df_good_data['ring_size'] < intensity_limit)]

radius_data = np.array(radius_data)
intensity_data = np.array(intensity_data)


In [ ]:
intensity_data_bin = []
radius_data_bin = []
intensity_data_bin_mean = []
intensity_data_bin_std = []
x_edges = []

xprev = 0.92
x_edges.append(xprev)
for i in range(0, 8):
    x = xprev + 0.04
    x_edges.append(x)

    intensity_data_bin.append(intensity_data[np.where((radius_data > xprev) & (radius_data <= x))[0]])
    radius_data_bin.append(radius_data[np.where((radius_data > xprev) & (radius_data <= x))[0]])
    intensity_data_bin_mean.append(np.mean(intensity_data[np.where((radius_data > xprev) & (radius_data <= x))[0]]))
    intensity_data_bin_std.append(np.std(intensity_data[np.where((radius_data > xprev) & (radius_data <= x))[0]]) / 
                                 np.sqrt(len(intensity_data[np.where((radius_data > xprev) & (radius_data <= x))[0]])))
    xprev = x 

x_centers = (np.array(x_edges[:-1]) + np.array(x_edges[1:]))/2

radius_data_fit = radius_data[(radius_data > x_edges[0]) & (radius_data <= x_edges[-1])]
intensity_data_fit = intensity_data[(radius_data > x_edges[0]) & (radius_data <= x_edges[-1])]
popt_data,pcov_data = curve_fit(func, radius_data_fit, intensity_data_fit)


### MC

In [ ]:
radius_mc = []
intensity_mc = []

for opt_eff in OPT_EFF:
    listdir=glob.glob(f'data/mc/trans_{opt_eff}/dl1*fits')
    dat = Table.read('{}'.format(listdir[0]), format='fits')
    for i in range(1,len(listdir)):
        dat2 = Table.read('{}'.format(listdir[i]), format='fits')
        dat = vstack([dat, dat2])
    df = dat.to_pandas()
    df_good = df[df['good_ring']]


    radius_mc.append(df_good['ring_radius'][(df_good['ring_completeness'] > 0.9) 
                                           & (df_good['size_outside'] < size_outside)
                                           & (df_good['ring_width'] < 0.4) & (df_good['ring_size'] < intensity_limit)])
    intensity_mc.append(df_good['ring_size'][(df_good['ring_completeness'] > 0.9) 
                                     & (df_good['size_outside'] < size_outside)
                                     & (df_good['ring_width'] < 0.4) & (df_good['ring_size'] < intensity_limit)])
    
radius_mc = np.array(radius_mc)
intensity_mc = np.array(intensity_mc)


In [ ]:
intensity_mc_bin_array = []
intensity_mc_bin_mean_array = []
intensity_mc_bin_std_array = []

popt_mc_array = []

for j in range(0, len(OPT_EFF)):

    intensity_mc_bin = []
    intensity_mc_bin_mean = []
    intensity_mc_bin_std = []
    x_edges = []

    xprev = 0.92
    x_edges.append(xprev)
    for i in range(0, 8):
        x = xprev + 0.04
        x_edges.append(x)

        intensity_mc_bin.append(np.array(intensity_mc[j])[np.where((radius_mc[j] > xprev) & (radius_mc[j] <= x))[0]])
        intensity_mc_bin_mean.append(np.mean(np.array(intensity_mc[j])[np.where((radius_mc[j] > xprev) & (radius_mc[j] <= x))[0]]))
        intensity_mc_bin_std.append(
            np.std(np.array(intensity_mc[j])[np.where((radius_mc[j] > xprev) & (radius_mc[j] <= x))[0]]) / np.sqrt(len(
        np.array(intensity_mc[j])[np.where((radius_mc[j] > xprev) & (radius_mc[j] <= x))[0]])))
        xprev = x 
        
    intensity_mc_bin_array.append(intensity_mc_bin)
    intensity_mc_bin_mean_array.append(intensity_mc_bin_mean)
    intensity_mc_bin_std_array.append(intensity_mc_bin_std)
    x_centers = (np.array(x_edges[:-1]) + np.array(x_edges[1:]))/2
    
    
    radius_mc_fit = radius_mc[j][(radius_mc[j] > x_edges[0]) & (radius_mc[j] <= x_edges[-1])]
    intensity_mc_fit = intensity_mc[j][(radius_mc[j] > x_edges[0]) & (radius_mc[j] <= x_edges[-1])]
    
    popt_mc,pcov_mc = curve_fit(func, radius_mc_fit, intensity_mc_fit)
    popt_mc_array.append(popt_mc)
    
    

In [ ]:
fig, ax = plt.subplots()

for i, (opt_eff, col, label, linestyle, facecolor) in enumerate(zip(OPT_EFF, colors, labels, linestyles, facecolors)):
    ax.errorbar(x_centers, intensity_mc_bin_mean_array[i], yerr=np.array(intensity_mc_bin_std_array[i]), ls='',
                 ms=10, fmt='o', markeredgecolor=col, color=facecolor, label = label)
    ax.errorbar(x_centers, intensity_mc_bin_mean_array[i], yerr=np.array(intensity_mc_bin_std_array[i]), ls='',
                 ms=10, fmt='', markeredgecolor=col, color=col)
    ax.plot(x_centers, popt_mc_array[i] * x_centers, color=col, ls=linestyle)

ax.errorbar(x_centers, intensity_data_bin_mean, yerr=intensity_data_bin_std, color = 'k',  
            label = 'Data (Run 2973)', ls='', fmt='o', ms=10)
ax.plot(x_centers, popt_data * x_centers, color='k')

ax.legend(ncol=1)
ax.set_xlabel('Muon ring radius [deg]')
ax.set_ylabel('Muon ring intensity [p.e.]')

!mkdir -p figures

fig.savefig('figures/intensity_vs_radius_scipy_errors.png')
fig.savefig('figures/intensity_vs_radius_scipy_errors.pdf')